In [1]:
import pandas as pd
x_train = pd.read_csv(r"C:\Users\brian\Desktop\JUNSOO\bigdata\data\x_train.csv" , encoding='cp949')
x_test = pd.read_csv(r"C:\Users\brian\Desktop\JUNSOO\bigdata\data\x_test.csv" , encoding='cp949')
y_train = pd.read_csv(r"C:\Users\brian\Desktop\JUNSOO\bigdata\data\y_train.csv" , encoding='cp949')


In [2]:
#필요없는 칼럼 지우기 
x_test_cust_id = x_test['cust_id'] #제출용 코드에는 cust id 필요 

x_test.drop(columns = ['cust_id'] , inplace = True)
x_train.drop(columns = ['cust_id'] , inplace = True)
y_train.drop(columns = ['cust_id'] , inplace = True)

#or 변수지정 ! (inplace 적용함수 : .drop , .fillna , .dropna , reset_index
#                               : sort_value 는 by 와 acending 만 ! inplace는 ... 흠)

print(x_train.head())

       총구매액     최대구매액       환불금액   주구매상품 주구매지점  내점일수   내점당구매건수    주말방문비율  구매주기
0  68282840  11264000  6860000.0      기타   강남점    19  3.894737  0.527027    17
1   2136000   2136000   300000.0     스포츠   잠실점     2  1.500000  0.000000     1
2   3197000   1639000        NaN  남성 캐주얼   관악점     2  2.000000  0.000000     1
3  16077620   4935000        NaN      기타   광주점    18  2.444444  0.318182    16
4  29050000  24000000        NaN      보석  본  점     2  1.500000  0.000000    85


In [3]:
# 결측치 처리
#x_test 환불금액 
#x_train 환불금액 결측치 존재

# fillna parameter 이용 or 데이터 시리즈 새로 정의
(x_test['환불금액'].fillna(0 , inplace = True))
x_train['환불금액'] = (x_train['환불금액']
                   .fillna(0))



C:\Users\brian\AppData\Local\Temp\ipykernel_21424\3640737384.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  (x_test['환불금액'].fillna(0 , inplace = True))


In [4]:
#범주형 변수를 인코딩하기
#print(x_train['주구매상품'].unique())
print(len(x_train['주구매지점'].unique())
      , (len(x_test['주구매상품'].unique()))
      ,sep = '\n')

# 원핫 인코딩을 한다면, 65개의 칼럼 생성됨. --> 라벨인코딩이 효율적!

from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

#train 데이터에 대해 라벨인코딩
x_train['주구매상품'] = encoder.fit_transform(x_train['주구매상품'])
print(x_train['주구매상품'].head(10))

#test 데이터에 대해 라벨인코딩
x_test['주구매상품'] = encoder.fit_transform(x_test['주구매상품'])
print(x_test['주구매상품'].head(10))

#train 및 test 데이터에 대해 라벨인코딩 - 주구매지점
x_train['주구매지점'] = encoder.fit_transform(x_train['주구매지점'])
x_test['주구매지점'] = encoder.fit_transform(x_test['주구매지점'])

print(encoder.classes_)

24
41
0     5
1    21
2     6
3     5
4    15
5    11
6    22
7    13
8     5
9     9
Name: 주구매상품, dtype: int32
0     3
1     9
2     0
3    31
4    19
5    40
6     5
7    20
8    40
9     0
Name: 주구매상품, dtype: int32
['강남점' '관악점' '광주점' '노원점' '대구점' '대전점' '동래점' '미아점' '본  점' '부산본점' '부평점' '분당점'
 '상인점' '센텀시티점' '안양점' '영등포점' '울산점' '인천점' '일산점' '잠실점' '전주점' '창원점' '청량리점'
 '포항점']


In [5]:
# 파생변수 만들기
# 환불금액이 0보다 크면 1 0과 같으면 0 으로 조건문 지정
condition = x_train['환불금액'] > 0

# condtion 변수를 사용하여 새로운 칼럼 형성 
x_train.loc[condition , '환불금액_new'] = 1
x_train.loc[~condition, '환불금액_new'] = 0
print(x_train[['환불금액' , '환불금액_new']]) # cf) .drop(columns = []) << [] 하나만 이용해도 된다. 안써도 된다?

# condition 이 잘 작동하므로 기존 칼럼 삭제

x_train.drop(columns = '환불금액' , inplace = True) ## 이미 수행된 결과로 인해 한번더 실행이 안됨 // 변수를 새로 저장?
x_train.head()


           환불금액  환불금액_new
0     6860000.0       1.0
1      300000.0       1.0
2           0.0       0.0
3           0.0       0.0
4           0.0       0.0
...         ...       ...
3495        0.0       0.0
3496  6049600.0       1.0
3497        0.0       0.0
3498        0.0       0.0
3499  5973000.0       1.0

[3500 rows x 2 columns]


,총구매액,최대구매액,주구매상품,주구매지점,내점일수,내점당구매건수,주말방문비율,구매주기,환불금액_new
0,68282840,11264000,5,0,19,3.894737,0.527027,17,1.0
1,2136000,2136000,21,19,2,1.500000,0.000000,1,1.0
2,3197000,1639000,6,1,2,2.000000,0.000000,1,0.0
3,16077620,4935000,5,2,18,2.444444,0.318182,16,0.0
4,29050000,24000000,15,8,2,1.500000,0.000000,85,0.0


In [6]:

# test 데이터셋도 똑같이 삭제
condition = x_test['환불금액'] > 0 
x_test.loc[condition, '환불금액_new'] = 1
x_test.loc[~condition, '환불금액_new'] = 0

print(x_test[['환불금액', '환불금액_new']])

x_test.drop(columns = '환불금액' , inplace = True)



            환불금액  환불금액_new
0      4050000.0       1.0
1     48034700.0       1.0
2     30521000.0       1.0
3            0.0       0.0
4            0.0       0.0
...          ...       ...
2477         0.0       0.0
2478         0.0       0.0
2479         0.0       0.0
2480         0.0       0.0
2481         0.0       0.0

[2482 rows x 2 columns]


In [7]:
x_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2482 entries, 0 to 2481
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   총구매액      2482 non-null   int64  
 1   최대구매액     2482 non-null   int64  
 2   주구매상품     2482 non-null   int32  
 3   주구매지점     2482 non-null   int32  
 4   내점일수      2482 non-null   int64  
 5   내점당구매건수   2482 non-null   float64
 6   주말방문비율    2482 non-null   float64
 7   구매주기      2482 non-null   int64  
 8   환불금액_new  2482 non-null   float64
dtypes: float64(3), int32(2), int64(4)
memory usage: 155.3 KB


In [8]:
# 표준화 크기로 변환하기
# Data Scaling
print(x_train.describe().T)

# 표준화 크기변환 라이브러리

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler() #scaler 객체생성

x_train = pd.DataFrame(scaler.fit_transform(x_train), columns = x_train.columns) # scler 객체로 표준화 크기변환을 수행, x_train 칼럼명이용
x_test = pd.DataFrame(scaler.fit_transform(x_test), columns = x_test.columns)

print(x_train.describe().T)

           count          mean           std         min           25%  \
총구매액      3500.0  9.191925e+07  1.635065e+08 -52421520.0  4.747050e+06   
최대구매액     3500.0  1.966424e+07  3.199235e+07  -2992000.0  2.875000e+06   
주구매상품     3500.0  1.461200e+01  1.301995e+01         0.0  5.000000e+00   
주구매지점     3500.0  1.073429e+01  5.636480e+00         0.0  8.000000e+00   
내점일수      3500.0  1.925371e+01  2.717494e+01         1.0  2.000000e+00   
내점당구매건수   3500.0  2.834963e+00  1.912368e+00         1.0  1.666667e+00   
주말방문비율    3500.0  3.072463e-01  2.897516e-01         0.0  2.729090e-02   
구매주기      3500.0  2.095829e+01  2.474868e+01         0.0  4.000000e+00   
환불금액_new  3500.0  3.442857e-01  4.752027e-01         0.0  0.000000e+00   

                   50%           75%           max  
총구매액      2.822270e+07  1.065079e+08  2.323180e+09  
최대구매액     9.837000e+06  2.296250e+07  7.066290e+08  
주구매상품     9.000000e+00  2.200000e+01  4.100000e+01  
주구매지점     9.000000e+00  1.500000e+01  2.300000e

In [9]:
# 상관관계 확인하기 (다중공산성의 문제 해결!)
# 금액이라는 의미가 포함된 칼럼중 상관관계가 높은 칼럼은 삭제할 것 
# ==> 상관계수가 0.6 이상이면 강한 상관관계

print(x_train[['총구매액','최대구매액','환불금액_new']].corr())
x_train = (x_train
           .drop(columns = '최대구매액'))
x_test.drop(columns = '최대구매액' , inplace = True)


              총구매액     최대구매액  환불금액_new
총구매액      1.000000  0.700080  0.403357
최대구매액     0.700080  1.000000  0.330687
환불금액_new  0.403357  0.330687  1.000000


In [13]:
#1.3 학습하고 평가하기
# 종속변수인 '성별' 칼럼을 분류하는 모델 // 의사결정나무 분류기를 활용

from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()    #model 객체 생성
model.fit(x_train, y_train)         #train set 으로 model 학습시키기
                                    #valueerror: number of labels=3500 does not match number of samples=2482
                                    # --> 독립변수와 종속변수의 라벨수가 다르다는 에러.. 처음 csv 읽을때 파일 잘못 가져옴!

y_test_predicted = model.predict(x_test)    #학습된 model을 활용하여 테스트 데이터의 종속변수를 예측하기
type(y_test_predicted)                      #dataframe 으로 타입을 변경할것
print(pd.DataFrame(y_test_predicted).head(3))


   0
0  1
1  0
2  0


In [ ]:
# 정확한 성능을 가진 모델을 만들려면 추가적인 정보를 입력해야함.
# 사람이 직접 추가정보를 전달하여 모델의 성능을 높임